## Linear Algebra Code


Here I have collected useful code for Module 1, Fall 2025

For now, it is just code for different versions of Gaussian Elimination. 



In [2]:
# Here are some imports which will be used in the code in the rest of the lab  

# Imports used for the code in CS 237

import numpy as np                # arrays and functions which operate on arrays

import matplotlib.pyplot as plt   # normal plotting

import warnings


# NOTE: You may not use any other libraries than those listed here without explicit permission.
# given an augmented matrix A, do G. Elimination and print out steps
# At the end, report whether is no, 1, or multiple solutions

# number of digits of precision to print out
prec = 4

########################################################################################################

# Just calculates the REF and reports if no solution
# Appropriate for problems Ax=b, create augmented matrix
# A:b and input it as parameter

def GaussianElimination(Ab,traceRowOps=False):
    
    print("\nRunning Gaussian Elimination on:\n")
    print(np.round(Ab, decimals=4))
    
    B = forwardElimination(Ab,traceRowOps)
    print("\nEchelon Form:\n")
    B1 = np.round(B, decimals=4) + np.zeros(B.shape)                  # To prevent printing -0.0 after rounding
    print(np.round(B1, decimals=4),"\n")
    
    if (noSolution(B)):
        print("No solution!")

########################################################################################################

# Same, but continues on to do back substitution
def GaussJordan(Ab,traceRowOps=False):            
    print("\nRunning Gauss-Jordan on:\n")
    print(np.round(Ab, decimals=4))
    
    B = forwardElimination(Ab,traceRowOps)
    print("\nEchelon Form:\n")
    B1 = np.round(B, decimals=4) + np.zeros(B.shape) 
    print(B1,"\n")
    
    if (noSolution(B)):
        print("No solution!")
    else:
        C = backwardSubstitute(B,traceRowOps)
        print("Reduced Echelon Form:\n")
        C1 = np.round(C, decimals=4) + np.zeros(C.shape)                      # adding -0.0 + 0 gives 0.0
        print(C1,"\n")

########################################################################################################

# Performs GaussJordan on array which is NOT augmented;
# Used in dependency tests, determining rank, etc. 

def reduce2RREF(A,traceRowOps1=False):
    
    print("\nReducing to RREF:\n")
    print(np.round(A, decimals=4))
    
    B = forwardElimination(A,traceRowOps1)
    print("\nEchelon Form:\n")
    B1 = np.round(B, decimals=4) + np.zeros(B.shape) 
    print(B1,"\n")
    

    C = backwardSubstitute(B,augmented=False, traceRowOps=traceRowOps1)
    print("Reduced Echelon Form:\n")
    C1 = np.round(C, decimals=4) + np.zeros(C.shape) 
    print(C1,"\n")
        

########################################################################################################

def forwardElimination(A,traceRowOps=False):
    
    A = (np.copy(A)).astype(float)
    
    if (traceRowOps):
        print("\nRunning Forward Elimination on:\n")
        print(np.round(A, decimals=4),"\n")
        print()
    
    if (traceRowOps):
        print("Creating Echelon Form...\n")
    
    (numRows,numCols) = A.shape
    
    # r = row we are currently working on         pivot value is A[r][c]
    r = 0            
    for c in range(numCols):     # solve for variable in column c 
        # find row in column c with first non-zero element, and exchange with row r                  
        r1 = r
        while(r1 < numRows):
            if (not np.isclose(A[r1][c],0.0)):   # A[r1][c] is first non-zero element at or below r in column c
                break
            r1 += 1
        
        if(r1 == numRows):    # all zeros below r in this column
          #  print('continue')
            continue          # go on to the next column, but still working on same row   
         
        if(r1 != r):
            exchangeRows(A, r1, r) 
            if (traceRowOps): 
                print("Exchange R" + str(r1+1) + " and R" + str(r+1) + "\n") 
                print(np.round(A, decimals=4))                
                print() 

        # now use pivot A[r][c] to eliminate all vars in this column below row r
        for r2 in range(r+1,numRows):
            eliminateVariable(A,r,c,r2,traceRowOps)
            
        r += 1  
        if (r >= numRows):
            break
            
    return A

# for pivot A[r][c], eliminate variable in location A[r2][c] in row r2 using row operations
def eliminateVariable(A,r,c,r2,traceRowOps=False):

    if(not np.isclose(A[r2][c],0.0)):

        factor = -A[r2][c]/A[r][c] 

        addRowMult( A, r2, r, factor )
            
        if(traceRowOps):
            print("R" + str(r2+1) + " += " + str(np.around(factor,prec)) + "*R" + str(r+1) + "\n")  
            print(np.round(A, decimals=4))
            print()


def backwardSubstitute(A,augmented=True,traceRowOps=False): 
    
    numRows,numCols = A.shape
    
    if (A.dtype != 'float64'):
        A = A.astype(float)

    # now back-substitute the variables from bottom row to top
    if (traceRowOps):
        print("Back Substituting....\n") 

    for r in range(numRows):

        # find variable in this row
        for c in range(numCols):
            if(not np.isclose(A[r][c],0.0)):
                break 
       
        #print(numCols, r, c)
        if (augmented and c >= numCols-1):        # inconsistent or redundant row
                continue
        elif (c >= numCols):                      # inconsistent or redundant row
                continue  
            
            
        # A[r][c] is variable to eliminate
        
        factor = A[r][c]
        
        if (np.isclose(factor,0.0)):
            continue
        
        if(not np.isclose(factor,1.0)):  
            multRowByScalar(A, r, 1/factor)
            if (traceRowOps):
                print("R" + str(r+1) + " = R" + str(r+1) + "/" + str(np.around(factor,prec)) + "\n")  
                print(np.round(A, decimals=4))
                print()

        for r2 in range(r): 
            eliminateVariable(A,r,c,r2,traceRowOps)
        


    return A 


def exchangeRows(A, r1, r2):
    tmp = A[r1].copy()
    A[r1] = A[r2]
    A[r2] = tmp

def multRowByScalar(A, r, s): 
    A[r] *= s

def addRowMult(A, r1, r2, s): 
    A[r1] += s * A[r2]
    
# try to find row of all zeros except for last column, in augmented matrix. 

def noSolution(A):
    numRows,numCols = A.shape
    for r in range(numRows-1,-1,-1):         # start from bottom, since inconsistent rows end up there
        for c in range(numCols):
            if(not np.isclose(A[r][c],0.0)):  # found first non-0 in this row
                if(c == numCols-1):
                    return True
                else:
                    break
    return False 

# determines if matrix is in form of augmented identity matrix

def uniqueSolution(A):
    # test shape first
    if(numRows != A[0].size - 1):
        return False 
    # then test if diagonal is all 1's and rest is all 0's
    for r in range(numRows):
        for c in range(numRows):
            if(r == c and not np.isclose(A[r][c],1.0)):
                return False
            elif(r != c and not np.isclose(A[r][c],0.0)):
                return False
    return True   


#### Example 1: (Unique solution)

As a **set of equations:**


$$
\begin{array}{rcrcrcl}
2x_1 & - & x_2 & + & 4x_3 & = & 1\\
x_1  &  + & 6x_2    & - & 4x_3 & = & 2\\
3x_1  & +  &  x_2   & + & 2x_3 & = & -1
\end{array}
$$

As a **matrix equation**: $\mathbf{A}\mathbf{x}=\mathbf{b}$

$$
\mathbf{A}=
\begin{bmatrix}
2 & -1 & 4\\
1 & 6 & -4\\
3 & 1 & 2
\end{bmatrix},\qquad
\mathbf{x}=
\begin{bmatrix}
x_1\\ x_2\\ x_3
\end{bmatrix},\qquad
\mathbf{b}=
\begin{bmatrix}
1\\ 2\\ -1
\end{bmatrix}
$$

As an **augmented matrix**:
$$
\left[
\begin{array}{ccc|c}
2 & -1 & 4 & 1\\
1 & 0  & -4& 2\\
1 & 0  & 2 & -1
\end{array}
\right].
$$

In [3]:
A = np.array([[2,-1,4], [1,6,-4], [3,1,2]])
b = np.array([[1,2,-1]]).T
print(A)
print(b)
Ab = np.hstack([A,b])
Ab

[[ 2 -1  4]
 [ 1  6 -4]
 [ 3  1  2]]
[[ 1]
 [ 2]
 [-1]]


array([[ 2, -1,  4,  1],
       [ 1,  6, -4,  2],
       [ 3,  1,  2, -1]])

Tracing the variable elimination:

In [4]:
GaussJordan(Ab,True)


Running Gauss-Jordan on:

[[ 2 -1  4  1]
 [ 1  6 -4  2]
 [ 3  1  2 -1]]

Running Forward Elimination on:

[[ 2. -1.  4.  1.]
 [ 1.  6. -4.  2.]
 [ 3.  1.  2. -1.]] 


Creating Echelon Form...

R2 += -0.5*R1

[[ 2.  -1.   4.   1. ]
 [ 0.   6.5 -6.   1.5]
 [ 3.   1.   2.  -1. ]]

R3 += -1.5*R1

[[ 2.  -1.   4.   1. ]
 [ 0.   6.5 -6.   1.5]
 [ 0.   2.5 -4.  -2.5]]

R3 += -0.3846*R2

[[ 2.     -1.      4.      1.    ]
 [ 0.      6.5    -6.      1.5   ]
 [ 0.      0.     -1.6923 -3.0769]]


Echelon Form:

[[ 2.     -1.      4.      1.    ]
 [ 0.      6.5    -6.      1.5   ]
 [ 0.      0.     -1.6923 -3.0769]] 

Reduced Echelon Form:

[[ 1.      0.      0.     -2.1818]
 [ 0.      1.      0.      1.9091]
 [ 0.      0.      1.      1.8182]] 



So, the **solution** is

$$x_1\ =\ -2.1818\qquad x_2\ =\ 1.9091\qquad x_3\ =\ 1.8182$$

**Solving with the matrix inverse:**

$$A\mathbf{x}\ = \ \mathbf{b}$$

$$A^{-1}A\mathbf{x}\ =\ A^{-1} \mathbf{b}$$

$$\mathbf{x}\ =\ A^{-1} \mathbf{b}$$

In [5]:
Ainv = np.linalg.inv(A)
Ainv

array([[-0.72727273, -0.27272727,  0.90909091],
       [ 0.63636364,  0.36363636, -0.54545455],
       [ 0.77272727,  0.22727273, -0.59090909]])

In [6]:
# Should produce identity matrix

A @ Ainv

array([[ 1.0000000e+00,  0.0000000e+00,  4.4408921e-16],
       [-4.4408921e-16,  1.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])

In [7]:
Ainv @ A

array([[ 1.00000000e+00,  4.44089210e-16,  4.44089210e-16],
       [-2.22044605e-16,  1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.22044605e-16,  1.00000000e+00]])

In [8]:
np.around(A @ Ainv,4)

array([[ 1.,  0.,  0.],
       [-0.,  1.,  0.],
       [ 0.,  0.,  1.]])

**Digression: How to check for equality with floating point matrices?**

In [9]:
I3 = np.eye(3)
I3

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [10]:
A @ Ainv == I3

array([[False,  True, False],
       [False, False,  True],
       [ True,  True, False]])

In [11]:
np.around(A @ Ainv,3) == I3

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [12]:
np.all(A @ Ainv == I3)

False

In [13]:
np.isclose(A @ Ainv,I3)

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

**Conclusion:**  Use `np.allclose` instead of `==` on matrices. 

In [14]:
np.allclose(A @ Ainv,I3)

True

**End of Digression**

**Solving using the matrix inverse**

In [16]:
Ainv = np.linalg.inv(A)

Ainv @ A

array([[ 1.00000000e+00,  4.44089210e-16,  4.44089210e-16],
       [-2.22044605e-16,  1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.22044605e-16,  1.00000000e+00]])

In [17]:
Ainv @ b

array([[-2.18181818],
       [ 1.90909091],
       [ 1.81818182]])

$$x_1\ =\ -2.1818\qquad x_2\ =\ 1.9091\qquad x_3\ =\ 1.8182$$

**Solving using `numpy.linalg.solve`**

In [18]:
x = np.linalg.solve(A,b)
x

array([[-2.18181818],
       [ 1.90909091],
       [ 1.81818182]])

In [19]:
# check

A @ x == b
# np.all(A @ x == b)

array([[False],
       [False],
       [False]])

In [20]:
np.allclose(A @ x,b)

True

### Equation set 2:  No solution

In [21]:
A = np.array([[2,-2,4], [1,-1,2], [3,1,2]])
b = np.array([[1,2,-1]]).T
print(A)
print(b)
Ab = np.hstack([A,b])
Ab
GaussianElimination(Ab,True)

[[ 2 -2  4]
 [ 1 -1  2]
 [ 3  1  2]]
[[ 1]
 [ 2]
 [-1]]

Running Gaussian Elimination on:

[[ 2 -2  4  1]
 [ 1 -1  2  2]
 [ 3  1  2 -1]]

Running Forward Elimination on:

[[ 2. -2.  4.  1.]
 [ 1. -1.  2.  2.]
 [ 3.  1.  2. -1.]] 


Creating Echelon Form...

R2 += -0.5*R1

[[ 2.  -2.   4.   1. ]
 [ 0.   0.   0.   1.5]
 [ 3.   1.   2.  -1. ]]

R3 += -1.5*R1

[[ 2.  -2.   4.   1. ]
 [ 0.   0.   0.   1.5]
 [ 0.   4.  -4.  -2.5]]

Exchange R3 and R2

[[ 2.  -2.   4.   1. ]
 [ 0.   4.  -4.  -2.5]
 [ 0.   0.   0.   1.5]]


Echelon Form:

[[ 2.  -2.   4.   1. ]
 [ 0.   4.  -4.  -2.5]
 [ 0.   0.   0.   1.5]] 

No solution!


In [23]:
# np.linalg.inv(A)

#### Multiple Solutions


In [24]:
A = np.array([[2,-2,4], [1,-1,2], [3,1,2]])
b = np.array([[2,1,-1]]).T
print(A)
print(b)
Ab = np.hstack([A,b])
Ab
GaussianElimination(Ab,True)

[[ 2 -2  4]
 [ 1 -1  2]
 [ 3  1  2]]
[[ 2]
 [ 1]
 [-1]]

Running Gaussian Elimination on:

[[ 2 -2  4  2]
 [ 1 -1  2  1]
 [ 3  1  2 -1]]

Running Forward Elimination on:

[[ 2. -2.  4.  2.]
 [ 1. -1.  2.  1.]
 [ 3.  1.  2. -1.]] 


Creating Echelon Form...

R2 += -0.5*R1

[[ 2. -2.  4.  2.]
 [ 0.  0.  0.  0.]
 [ 3.  1.  2. -1.]]

R3 += -1.5*R1

[[ 2. -2.  4.  2.]
 [ 0.  0.  0.  0.]
 [ 0.  4. -4. -4.]]

Exchange R3 and R2

[[ 2. -2.  4.  2.]
 [ 0.  4. -4. -4.]
 [ 0.  0.  0.  0.]]


Echelon Form:

[[ 2. -2.  4.  2.]
 [ 0.  4. -4. -4.]
 [ 0.  0.  0.  0.]] 



In [25]:
# RRREF


rref = np.array([[ 1, -1,  2,  1],
 [ 0,  1, -1, -1],
 [ 0,  0,  0,  0]] )

rref

array([[ 1, -1,  2,  1],
       [ 0,  1, -1, -1],
       [ 0,  0,  0,  0]])

$$
\begin{aligned}
x_1 - x_2 + 2x_3 &= 1,\\
\phantom{x_1}  x_2 - x_3 &= -1,\\
0x_1 + 0x_2 + 0x_3 &= 0.
\end{aligned}
$$


**Note:**  Free variable $x_3$ can be **anything** and other variables are defined by the free variables!

$$ x_1\ =\ 2 - x_3$$
$$x_2\ =\ x_3 - 1$$


- **One free variable:**  Solution is a line
- **Two free variables:** Solution is a plane